In [1]:
import os
import pandas as pd 
import numpy as np

In [2]:
train = pd.read_csv('./数据集/科创企业成长潜力模型/label/train_2.csv')
train.rename(columns={'label':'Y'}, inplace = True)
test = pd.read_csv('./数据集/科创企业成长潜力模型/upload.csv')

data_all = pd.concat((train[['ID','Y']],test[['ID','Y']]))

In [3]:
data_all

,ID,Y
0,corp_1001749,0.0
1,corp_1001231,0.0
2,corp_1004522,0.0
3,corp_1001738,1.0
4,corp_1003948,1.0
...,...,...
517,corp_1000552,NaN
518,corp_1001079,NaN
519,corp_1004730,NaN
520,corp_1000350,NaN


In [4]:
data_all['ID'].nunique()

4959

In [5]:
folder_path = './数据集/科创企业成长潜力模型/data'  # 替换为你的文件夹路径
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx') and file.startswith('政府支持') ]

In [6]:
len(excel_files)

25

In [7]:
excel_files

['政府支持_产品技术_再制造产品.xlsx',
 '政府支持_产品技术_升级和创新消费品.xlsx',
 '政府支持_产品技术_国家成熟适用节水技术推广.xlsx',
 '政府支持_产品技术_国家鼓励重大环保技术装备.xlsx',
 '政府支持_产品技术_水污染防治技术.xlsx',
 '政府支持_产品技术_绿色数据中心适用产品.xlsx',
 '政府支持_产品技术_能效之星产品.xlsx',
 '政府支持_产品技术_认定新技术新产品.xlsx',
 '政府支持_产品技术_首台重大技术装备.xlsx',
 '政府支持_培育项目_上市后备和培育企业.xlsx',
 '政府支持_培育项目_中国独角兽企业.xlsx',
 '政府支持_培育项目_政府支持其他培育企业.xlsx',
 '政府支持_培育项目_政府重点支持其他项目企业.xlsx',
 '政府支持_培育项目_政府重点试点项目企业.xlsx',
 '政府支持_培育项目_隐形冠军.xlsx',
 '政府支持_培育项目_雏鹰企业.xlsx',
 '政府支持_资金支持_中央引导地方科技发展资金支持项目.xlsx',
 '政府支持_资金支持_中小企业发展专项资金支持项目.xlsx',
 '政府支持_资金支持_产业结构调整专项资金支持项目.xlsx',
 '政府支持_资金支持_地方政府专项资金奖补企业.xlsx',
 '政府支持_资金支持_工业与信息化发展专项资金支持项目.xlsx',
 '政府支持_资金支持_工业转型升级专项资金支持项目.xlsx',
 '政府支持_资金支持_技术改造专项资金支持项目.xlsx',
 '政府支持_资金支持_科技创新券补贴企业.xlsx',
 '政府支持产品技术_政府认定各类领先产品.xlsx']

In [8]:
def reform_mutiindex(df):
    level0 = df.columns.get_level_values(0)
    level1 = df.columns.get_level_values(1)
    df.columns = level0 + '_' + level1
    df.rename(columns={'ID_':'ID'}, inplace = True)
    return df

In [9]:
# 
df1 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[0],index_col=0)
df1.rename(columns={'企业名称':'ID'}, inplace = True)
df1['再制造产品_级别'] = df1['级别'].map({'国家级': 2, '省级': 1})
df1['再制造产品_类型'] = df1['类型'].map({'认定': 3, '拟认定': 2, '通过验收': 1})
df1['再制造产品_发布日期'] = pd.to_datetime(df1['发布日期'])
df1['再制造产品_发布日期'] = (pd.to_datetime('2024-04-30') - df1['再制造产品_发布日期']).dt.days
df1 = df1.groupby('ID').agg({'再制造产品_级别':['sum'], '再制造产品_类型': ['sum'], '再制造产品_发布日期': ['min','max','count']}).reset_index().fillna(0)

df1 = reform_mutiindex(df1)
data_all = data_all.merge(df1,on='ID',how='left')

In [10]:
df2 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[1],index_col=0)
df2 = df2.fillna(0)
df2.rename(columns={'企业名称':'ID'}, inplace = True)
df2['升级和创新消费品_类型'] = df2['类型'].map({'创新消费品': 2, '升级消费品': 1})
df2['升级和创新消费品_产品类别'] = df2['产品类别'].map({'五金': 1, '0': 0, '锅具':1,'乳制品':2,
                                        '制笔':3,'吸油烟机':1,'燃气灶具':1,'热水器':1,'婴童用品':4,
                                        '安全座椅':4,'童车':4,'塑料制品':5,'日用五金':1,'家具':6})
df2['升级和创新消费品_发布日期'] = pd.to_datetime(df2['发布日期'])
df2['升级和创新消费品_发布日期'] = (pd.to_datetime('2024-04-30') - df2['升级和创新消费品_发布日期']).dt.days

df2 = df2.groupby('ID').agg({'升级和创新消费品_类型':['mean'], '升级和创新消费品_产品类别': ['mean'], '升级和创新消费品_发布日期': ['min','max','count']}).reset_index().fillna(0)
df2 = reform_mutiindex(df2)

data_all = data_all.merge(df2,on='ID',how='left')

In [11]:
df3 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[2],index_col=0)
df3.rename(columns={'原名单公司现名称':'ID','节水技术领域':'节水技术'}, inplace = True)
df3['节水技术'] = 1
df3 = df3[['ID','节水技术']]
df3 = df3.groupby('ID').agg({'节水技术':['mean','count']}).reset_index().fillna(0)
df3 = reform_mutiindex(df3)

data_all = data_all.merge(df3,on='ID',how='left')

In [12]:
df4 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[3],index_col=0)
df4.rename(columns={'企业名称':'ID'}, inplace = True)
df4 = df4.merge(data_all,on='ID',how='left')
df4['国家鼓励重大环保技术装备_应用范围'] = df4.groupby('应用范围')['Y'].transform('mean').fillna(0)
df4['国家鼓励重大环保技术装备_典型案例类型'] = df4.groupby('典型案例类型')['Y'].transform('mean').fillna(0)
df4 = df4.groupby('ID').agg({'国家鼓励重大环保技术装备_应用范围':['mean','count'],'国家鼓励重大环保技术装备_典型案例类型':'mean'}).reset_index().fillna(0)
df4 = reform_mutiindex(df4)
data_all = data_all.merge(df4,on='ID',how='left')

In [13]:
df5 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[4],index_col=0)
df5.rename(columns={'企业名称':'ID'}, inplace = True)
df5['水污染防治技术'] = 1
df5 = df5[['ID','水污染防治技术']]
df5 = df5.groupby('ID').agg({'水污染防治技术':['mean','count']}).reset_index().fillna(0)
df5 = reform_mutiindex(df5)

data_all = data_all.merge(df5,on='ID',how='left')

In [14]:
df6 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[5],index_col=0)
df6.rename(columns={'企业名称':'ID'}, inplace = True)
df6['绿色数据中心适用产品'] = 1
df6 = df6[['ID','绿色数据中心适用产品']]
df6 = df6.groupby('ID').agg({'绿色数据中心适用产品':['mean','count']}).reset_index().fillna(0)
df6 = reform_mutiindex(df6)

data_all = data_all.merge(df6,on='ID',how='left')

In [15]:
df7 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[7],index_col=0)
df7.rename(columns={'企业名称':'ID'}, inplace = True)
df7 = df7.merge(data_all,on='ID',how='left')
df7['认定新技术新产品_类型'] = df7.groupby('类型')['Y'].transform('mean').fillna(0)
df7 = df7[['ID','认定新技术新产品_类型']]
df7 = df7.groupby('ID').agg({'认定新技术新产品_类型':['mean','count']}).reset_index().fillna(0)
df7 = reform_mutiindex(df7)

data_all = data_all.merge(df7,on='ID',how='left')

In [16]:
df8 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[8],index_col=0)
df8.rename(columns={'企业名称':'ID','级别':'首台重大技术装备_级别','类型':'首台重大技术装备_类型',
                    '发布日期':'首台重大技术装备_发布日期'}, inplace = True)

df8 = df8.merge(data_all,on='ID',how='left')
df8['首台重大技术装备_级别'] = df8.groupby('首台重大技术装备_级别')['Y'].transform('mean').fillna(0)
df8['首台重大技术装备_类型'] = df8.groupby('首台重大技术装备_类型')['Y'].transform('mean').fillna(0)
df8['首台重大技术装备_发布日期'] = pd.to_datetime(df8['首台重大技术装备_发布日期'])
df8['首台重大技术装备_发布日期'] = (pd.to_datetime('2024-04-30') - df8['首台重大技术装备_发布日期']).dt.days

df8 = df8[['ID','首台重大技术装备_级别','首台重大技术装备_类型','首台重大技术装备_发布日期']]
df8 = df8.groupby('ID').agg({'首台重大技术装备_级别':['sum'],'首台重大技术装备_类型':['sum'],
                             '首台重大技术装备_发布日期':['min','max','count']}).reset_index().fillna(0)
df8 = reform_mutiindex(df8)

data_all = data_all.merge(df8,on='ID',how='left')

In [17]:
df9 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[9],index_col=0)
df9.rename(columns={'企业名称':'ID'}, inplace = True)
df9['上市后备和培育企业'] = 1
df9 = df9[['ID','上市后备和培育企业']]
df9 = df9.groupby('ID').agg({'上市后备和培育企业':['mean','count']}).reset_index().fillna(0)
df9 = reform_mutiindex(df9)

data_all = data_all.merge(df9,on='ID',how='left')

In [18]:
df10 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[10],index_col=0)
df10.rename(columns={'企业名称':'ID'}, inplace = True)
# 找到每个企业的年份列中估值最大的年份
max_years = df10.groupby('ID')['认定年度'].idxmax()
# 提取对应的企业估值
df10 = df10.loc[max_years, ['ID', '企业估值']]

data_all = data_all.merge(df10,on='ID',how='left')

In [19]:
df11 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[11],index_col=0)
df11.rename(columns={'公司名':'ID','年度':'政府支持其他培育企业_年度'}, inplace = True)
df11['政府支持其他培育企业'] = 1
df11 = df11.groupby('ID').agg({'政府支持其他培育企业':['sum'], '政府支持其他培育企业_年度': ['min','max','count']}).reset_index().fillna(0)
df11 = reform_mutiindex(df11)

data_all = data_all.merge(df11,on='ID',how='left')

In [20]:
df12 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[12],index_col=0)
df12.rename(columns={'企业名称':'ID','年度':'政府重点支持其他项目企业_年度','级别':'政府重点支持其他项目企业_级别'}, inplace = True)
df12 = df12.merge(data_all,on='ID',how='left')
df12['政府重点支持其他项目企业_级别'] = df12.groupby('政府重点支持其他项目企业_级别')['Y'].transform('mean').fillna(0)


df12 = df12[['ID','政府重点支持其他项目企业_年度','政府重点支持其他项目企业_级别']]
df12 = df12.groupby('ID').agg({'政府重点支持其他项目企业_级别':['sum'],'政府重点支持其他项目企业_年度':['min','max','count']}).reset_index().fillna(0)
df12 = reform_mutiindex(df12)

data_all = data_all.merge(df12,on='ID',how='left')

In [21]:
df13 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[13],index_col=0)
df13.rename(columns={'企业名称':'ID','年度':'政府重点试点项目企业_年度','级别':'政府重点试点项目企业_级别'}, inplace = True)
df13 = df13.merge(data_all,on='ID',how='left')
df13['政府重点试点项目企业_级别'] = df13.groupby('政府重点试点项目企业_级别')['Y'].transform('mean').fillna(0)
df13 = df13[['ID','政府重点试点项目企业_年度','政府重点试点项目企业_级别']]

df13 = df13.groupby('ID').agg({'政府重点试点项目企业_级别':['sum'],'政府重点试点项目企业_年度':['min','max','count']}).reset_index().fillna(0)
df13 = reform_mutiindex(df13)

data_all = data_all.merge(df13,on='ID',how='left')

In [22]:
df14 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[14],index_col=0)
df14.rename(columns={'企业名称':'ID','级别':'隐形冠军_级别','年度':'隐形冠军_年度','名目':'隐形冠军_名目'}, inplace = True)
max_years = df14.groupby('ID')['隐形冠军_年度'].idxmax()
df14 = df14.loc[max_years, ['ID', '隐形冠军_级别','隐形冠军_年度','隐形冠军_名目']]
df14['隐形冠军_级别'] = df14['隐形冠军_级别'].map({'省级': 2, '市级': 1})
df14 = df14.merge(data_all,on='ID',how='left')
df14['隐形冠军_名目'] = df14.groupby('隐形冠军_名目')['Y'].transform('mean').fillna(0)
df14 = df14[['ID','隐形冠军_级别','隐形冠军_名目']]
df14 = df14.groupby('ID').agg({'隐形冠军_级别':'sum','隐形冠军_名目':'sum'}).reset_index().fillna(0)
data_all = data_all.merge(df14,on='ID',how='left')

In [23]:
df15 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[15],index_col=0)
max_years = df15.groupby('公司现用名')['年度'].idxmax()
df15 = df15.loc[max_years, :]
df15.rename(columns={'公司现用名':'ID','类型':'雏鹰企业_类型','行业':'雏鹰企业_行业'}, inplace = True)
df15 = df15.merge(data_all,on='ID',how='left')
df15['雏鹰企业_类型'] = df15.groupby('雏鹰企业_类型')['Y'].transform('mean').fillna(0)
df15['雏鹰企业_行业'] = df15.groupby('雏鹰企业_行业')['Y'].transform('mean').fillna(0)
df15 = df15[['ID','雏鹰企业_类型','雏鹰企业_行业']]
df15 = df15.groupby('ID').agg({'雏鹰企业_类型':'sum','雏鹰企业_行业':'sum'}).reset_index().fillna(0)
data_all = data_all.merge(df15,on='ID',how='left')

In [24]:
df16 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[16],index_col=0)
df16.rename(columns={'企业名称':'ID','级别':'中央引导地方科技发展资金支持项目_级别','认定年度':'中央引导地方科技发展资金支持项目_认定年度'}, inplace = True)
df16 = df16.merge(data_all,on='ID',how='left')
df16['中央引导地方科技发展资金支持项目_级别'] = df16.groupby('中央引导地方科技发展资金支持项目_级别')['Y'].transform('mean').fillna(0)
df16 = df16.groupby('ID').agg({'中央引导地方科技发展资金支持项目_级别':'sum'}).reset_index().fillna(0)

data_all = data_all.merge(df16,on='ID',how='left')

In [25]:
df17 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[17],index_col=0)
df17.rename(columns={'企业名称':'ID','等级':'中小企业发展专项资金支持项目_等级'}, inplace = True)
df17 = df17.merge(data_all,on='ID',how='left')
df17['中小企业发展专项资金支持项目_等级'] = df17.groupby('中小企业发展专项资金支持项目_等级')['Y'].transform('mean').fillna(0)
df17 = df17.groupby('ID').agg({'中小企业发展专项资金支持项目_等级':'sum'}).reset_index().fillna(0)

data_all = data_all.merge(df17,on='ID',how='left')

In [26]:
df18 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[18],index_col=0)
df18.rename(columns={'公司名':'ID','等级':'产业结构调整专项资金支持项目_等级'}, inplace = True)
df18['产业结构调整专项资金支持项目_等级'] = 1
df18 = df18[['ID','产业结构调整专项资金支持项目_等级']]

data_all = data_all.merge(df18,on='ID',how='left')

In [27]:
df19 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[19],index_col=0)
df19.rename(columns={'企业名称':'ID','等级':'地方政府专项资金奖补企业_等级'}, inplace = True)
df19['地方政府专项资金奖补企业_等级'] = 1
df19 = df19.groupby('ID').agg({'地方政府专项资金奖补企业_等级':'sum'}).reset_index().fillna(0)

data_all = data_all.merge(df19,on='ID',how='left')

In [28]:
df20 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[20],index_col=0)
df20.rename(columns={'企业名称':'ID','级别':'工业与信息化发展专项资金支持项目_级别'}, inplace = True)
df20 = df20.groupby('ID').agg({'工业与信息化发展专项资金支持项目_级别':'count'}).reset_index().fillna(0)

data_all = data_all.merge(df20,on='ID',how='left')

In [29]:
df21 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[21],index_col=0)
df21.rename(columns={'企业名称':'ID','级别':'工业转型升级专项资金支持项目_级别'}, inplace = True)
df21['工业转型升级专项资金支持项目_级别'] = df21['工业转型升级专项资金支持项目_级别'].map({'省级': 2, '市级': 1})
df21 = df21.groupby('ID').agg({'工业转型升级专项资金支持项目_级别':'sum'}).reset_index().fillna(0)

data_all = data_all.merge(df21,on='ID',how='left')

In [30]:
def clean_22(x):
    if '省级' in x:
        return 2
    elif '市级' in x:
        return 1
    else:
        return 0

df22 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[22],index_col=0)
df22.rename(columns={'企业名称':'ID','级别':'技术改造专项资金支持项目_级别'}, inplace = True)
df22['工业转型升级专项资金支持项目_级别'] = df22['技术改造专项资金支持项目_级别'].apply(clean_22)
df22 = df22.groupby('ID').agg({'工业转型升级专项资金支持项目_级别':'sum'}).reset_index().fillna(0)

data_all = data_all.merge(df22,on='ID',how='left')

In [31]:
df23 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[23],index_col=0)
df23.rename(columns={'企业名称':'ID','级别':'科技创新券补贴企_级别'}, inplace = True)
df23['科技创新券补贴企_级别'] = 1
df23 = df23.groupby('ID').agg({'科技创新券补贴企_级别':'sum'}).reset_index().fillna(0)

data_all = data_all.merge(df23,on='ID',how='left')

In [32]:
df24 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[24],index_col=0)
df24.rename(columns={'企业名称':'ID','级别':'政府认定各类领先产品_级别','类型':'政府认定各类领先产品_类型'}, inplace = True)
df24['政府认定各类领先产品_级别'] = 1

df24 = df24.merge(data_all,on='ID',how='left')
df24['政府认定各类领先产品_类型'] = df24.groupby('政府认定各类领先产品_类型')['Y'].transform('mean').fillna(0)
df24 = df24.groupby('ID').agg({'政府认定各类领先产品_级别':'sum','政府认定各类领先产品_类型':'sum'}).reset_index().fillna(0)
df24 = df24[['ID','政府认定各类领先产品_级别','政府认定各类领先产品_类型']]

data_all = data_all.merge(df23,on='ID',how='left')

In [33]:
data_all

,ID,Y,再制造产品_级别_sum,再制造产品_类型_sum,再制造产品_发布日期_min,再制造产品_发布日期_max,再制造产品_发布日期_count,升级和创新消费品_类型_mean,升级和创新消费品_产品类别_mean,升级和创新消费品_发布日期_min,...,雏鹰企业_行业,中央引导地方科技发展资金支持项目_级别,中小企业发展专项资金支持项目_等级,产业结构调整专项资金支持项目_等级,地方政府专项资金奖补企业_等级,工业与信息化发展专项资金支持项目_级别,工业转型升级专项资金支持项目_级别_x,工业转型升级专项资金支持项目_级别_y,科技创新券补贴企_级别_x,科技创新券补贴企_级别_y
0,corp_1001749,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,corp_1001231,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.730769,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,corp_1004522,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,corp_1001738,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,corp_1003948,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5017,corp_1000552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5018,corp_1001079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5019,corp_1004730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.510204,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5020,corp_1000350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [34]:
data_all.to_csv('gov.csv',index=0)